In [13]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player

In [14]:
from GreedyPlayer import GreedyPlayer # greedy
from GreedyPlayer2 import GreedyPlayer2 # greedy player 2
from LowCardThrower import LowCardThrower # lowest first
from AggresivePlayer import AggresivePlayer # basic tracking
from NaiveHistoryTracker import NaiveHistoryTracker
from Vasyl import Vasia
from BalancedBluffer import BalancedBluffer
from Wiktor import Wiktor

from card_counter import CardCounter, CardCounter2, CardCounter3 # Player tracking history
from czech_nagorka import Czech_Nagorka

In [15]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [16]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (9, 3), (11, 3), (13, 0), (12, 3), (13, 3), (10, 3), (13, 2)]
[(9, 0), (12, 1), (13, 1), (11, 0), (14, 1), (12, 0), (10, 0), (14, 3)]


In [17]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [18]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [19]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["GreedyPlayer1", "GREEDY1", GreedyPlayer],
            ["GreedyPlayer2", "GREEDY2", GreedyPlayer2], 
            ["LowestFirst", "LowestFirst", LowCardThrower], 
            ["Aggresive", "AGGRESIVE", AggresivePlayer],
            ["NaiveTracker", "NaiveTracker", NaiveHistoryTracker],
            ["BalancedBluffer", "BalancedBluffer", BalancedBluffer],
            ["Vasyl", "Vasyl", Vasia],
            ["CardCounter", "CardCounter", CardCounter],
            ["CardCounter2", "CardCounter2", CardCounter2],
            ["CardCounter3", "CardCounter3", CardCounter3],
            ["Czech_Nagorka", "Czech_Nagorka", Czech_Nagorka],
            ["Wiktor", "Wiktor", Wiktor]
           ]

In [20]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results

==== 0 RandomPlayer
1 GreedyPlayer1
2 GreedyPlayer2
3 LowestFirst
4 Aggresive
5 NaiveTracker
6 BalancedBluffer
7 Vasyl
8 CardCounter
9 CardCounter2
10 CardCounter3
11 Czech_Nagorka
12 Wiktor
==== 1 GreedyPlayer1
2 GreedyPlayer2
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DR

In [21]:
full_results

[[None,
  [[0, 1000],
   [0, 0],
   [15.55, 16.064],
   [15.383, 0.0],
   0.617,
   [7.969, 12.989],
   [0.0, 0.0],
   [8.77, 0.0],
   [0, 0],
   0,
   0],
  [[15, 985],
   [0, 0],
   [20.975, 21.448],
   [14.865, 0.214],
   0.921,
   [10.729, 11.209],
   [0.004, 0.0],
   [11.908, 10.935],
   [0, 0],
   0,
   0],
  [[75, 925],
   [0, 0],
   [13.295, 13.703],
   [13.908, 0.973],
   1.119,
   [6.784, 5.711],
   [0.023, 0.0],
   [7.706, 2.863],
   [0, 0],
   0,
   0],
  [[0, 1000],
   [0, 0],
   [16.133, 16.62],
   [15.459, 0.0],
   0.541,
   [8.37, 13.582],
   [0.0, 0.0],
   [8.999, 0.0],
   [0, 0],
   0,
   0],
  [[0, 1000],
   [0, 0],
   [13.091, 13.598],
   [15.501, 0.0],
   0.499,
   [6.768, 9.639],
   [0.001, 0.0],
   [7.518, 0.0],
   [0, 0],
   0,
   0],
  [[1, 999],
   [0, 0],
   [10.329, 10.816],
   [14.401, 0.006],
   1.593,
   [5.408, 4.497],
   [0.0, 0.0],
   [5.839, 0.0],
   [0, 0],
   0,
   0],
  [[0, 1000],
   [0, 0],
   [10.21, 10.699],
   [14.344, 0.0],
   1.656,
   [5.38

Simple stats

In [22]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(f"  {A}")
        S = "  " + str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)

Wins
 RANDOM GREEDY1 GREEDY2 LowestFirst AGGRESIVE NaiveTracker BalancedBluffer Vasyl CardCounter CardCounter2 CardCounter3 Czech_Nagorka Wiktor 
  0
  RANDOM - 0 15 75 0 0 1 0 1 1 1 1 138 
  1
  GREEDY1 1000 - 904 949 495 1000 0 1 0 0 3 1 635 
  2
  GREEDY2 985 96 - 305 81 315 40 40 66 47 80 92 254 
  3
  LowestFirst 925 51 695 - 54 743 114 122 158 118 252 256 153 
  4
  AGGRESIVE 1000 505 919 946 - 1000 0 0 1 0 1 3 648 
  5
  NaiveTracker 1000 0 685 257 0 - 0 0 0 0 0 2 53 
  6
  BalancedBluffer 999 1000 960 886 1000 1000 - 490 600 563 585 496 194 
  7
  Vasyl 1000 999 960 878 1000 1000 510 - 623 561 589 493 232 
  8
  CardCounter 999 1000 934 842 999 1000 400 377 - 408 445 394 219 
  9
  CardCounter2 999 1000 953 882 1000 1000 437 439 592 - 569 460 214 
  10
  CardCounter3 999 997 920 748 999 1000 415 411 555 431 - 410 218 
  11
  Czech_Nagorka 999 999 908 744 997 998 504 507 606 540 590 - 250 
  12
  Wiktor 862 365 746 847 352 947 806 768 781 786 782 750 - 
Draws
 RANDOM GREEDY1 GRE

In [23]:
from copy import deepcopy

full_res = deepcopy(full_results)

N = 13
for i in range(N):
    for j in range(N):
        if i > j:
            full_res[i][j] = 1000 - full_res[j][i]
        elif i == j:
            full_res[i][j] = 0
        else:
            full_res[i][j] = full_res[i][j][0][0]

for i in range(N):
    for j in range(N):
        cell = full_res[i][j]
        if cell is not None:
            print(cell, end = ' ')
        else:
            print(None, end = ' ')
    print()

0 0 15 75 0 0 1 0 1 1 1 1 138 
1000 0 904 949 495 1000 0 1 0 0 3 1 635 
985 96 0 305 81 315 40 40 66 47 80 92 254 
925 51 695 0 54 743 114 122 158 118 252 256 153 
1000 505 919 946 0 1000 0 0 1 0 1 3 648 
1000 0 685 257 0 0 0 0 0 0 0 2 53 
999 1000 960 886 1000 1000 0 490 600 563 585 496 194 
1000 999 960 878 1000 1000 510 0 623 561 589 493 232 
999 1000 934 842 999 1000 400 377 0 408 445 394 219 
999 1000 953 882 1000 1000 437 439 592 0 569 460 214 
999 997 920 748 999 1000 415 411 555 431 0 410 218 
999 999 908 744 997 998 504 507 606 540 590 0 250 
862 365 746 847 352 947 806 768 781 786 782 750 0 


In [24]:
ranking = []
for i in range(N):
    ranking.append([strategy[i][0], sum(full_res[i])])
ranking.sort(key = lambda x: x[1], reverse=True)
for name, score in ranking:
    print(f"{name}: {score}")

Vasyl: 8845
Wiktor: 8792
BalancedBluffer: 8773
Czech_Nagorka: 8642
CardCounter2: 8545
CardCounter3: 8103
CardCounter: 8017
Aggresive: 5023
GreedyPlayer1: 4988
LowestFirst: 3641
GreedyPlayer2: 2401
NaiveTracker: 1997
RandomPlayer: 233
